In [1]:
!pip install -U transformers
!pip install -q gradio
!pip install transformers[torch]
!pip install tf-keras

In [2]:
!pip show transformers

Name: transformers
Version: 4.45.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: C:\Users\maria\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [3]:
import gradio as gr
import pandas as pd

## Analisando Dataset

Colunas Originais do Dataset: Index(['Unnamed: 0', 'title', 'ingredients', 'directions', 'link', 'source',
       'NER'],
      dtype='object')

Quantidade de Linhas Originalmente: 2231143

In [4]:
df = pd.read_csv("datasets/full_dataset.csv", engine='python')
print(df.head())

                                                 NER  \
0  ["brown sugar", "milk", "vanilla", "nuts", "bu...   
1  ["beef", "chicken breasts", "cream of mushroom...   
2  ["frozen corn", "cream cheese", "butter", "gar...   
3  ["chicken", "chicken gravy", "cream of mushroo...   
4  ["peanut butter", "graham cracker crumbs", "bu...   

                                          directions  
0  ["In a heavy 2-quart saucepan, mix brown sugar...  
1  ["Place chipped beef on bottom of baking dish....  
2  ["In a slow cooker, combine all ingredients. C...  
3  ["Boil and debone chicken.", "Put bite size pi...  
4  ["Combine first four ingredients and press in ...  


In [5]:
print(df.columns)

Index(['NER', 'directions'], dtype='object')


In [6]:
df["NER"].head()

0    ["brown sugar", "milk", "vanilla", "nuts", "bu...
1    ["beef", "chicken breasts", "cream of mushroom...
2    ["frozen corn", "cream cheese", "butter", "gar...
3    ["chicken", "chicken gravy", "cream of mushroo...
4    ["peanut butter", "graham cracker crumbs", "bu...
Name: NER, dtype: object

In [7]:
df = df[['NER', 'directions']]
df = df.dropna()

df.head()

,NER,directions
0,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu...","[""In a heavy 2-quart saucepan, mix brown sugar..."
1,"[""beef"", ""chicken breasts"", ""cream of mushroom...","[""Place chipped beef on bottom of baking dish...."
2,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar...","[""In a slow cooker, combine all ingredients. C..."
3,"[""chicken"", ""chicken gravy"", ""cream of mushroo...","[""Boil and debone chicken."", ""Put bite size pi..."
4,"[""peanut butter"", ""graham cracker crumbs"", ""bu...","[""Combine first four ingredients and press in ..."


In [8]:
# # shutill para backup antes de sobreencrever o arquivo
# import shutil

# shutil.copy("datasets/full_dataset.csv", "datasets/full_dataset_backup.csv")

# # Write the updated DataFrame back to the CSV file
# df.to_csv("datasets/full_dataset.csv", index=False)

In [9]:
# Step 2: Select the first 270,000 rows
# df = df.head(270000)

# Write the updated DataFrame back to the CSV file
# df.to_csv("datasets/full_dataset.csv", index=False)

## Tokenização

In [10]:
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [11]:
# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set pad_token to eos_token

In [12]:
# Define a function to tokenize the data
def tokenize_function(examples):
    inputs = [ex for ex in examples['NER']]
    targets = [ex for ex in examples['directions']]
    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True)
    
    # Tokenize the target text
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, padding="max_length", truncation=True)
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Convert DataFrame to Hugging Face Dataset format
dataset = Dataset.from_pandas(df)

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Parameter 'function'=<function tokenize_function at 0x0000025A84A1B4C0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
  0%|          | 0/270 [00:00<?, ?ba/s]C:\Users\maria\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if yo

## Fine-tuning

In [13]:
from transformers import TrainingArguments, Trainer

In [44]:
import os

# Create directories if they don't exist
if not os.path.exists('./logs'):
    os.makedirs('./logs')

if not os.path.exists('./results'):
    os.makedirs('./results')

In [46]:
# Load the GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Resize token embeddings to match the tokenizer

log_dir = os.path.abspath('./logs')
results_dir = os.path.abspath('./results')

# # Define training arguments
training_args = TrainingArguments(
    output_dir=results_dir,          # Output directory
    evaluation_strategy="epoch",     # Evaluate every epoch
    learning_rate=2e-5,
    per_device_train_batch_size=2,   # Batch size for training
    per_device_eval_batch_size=2,    # Batch size for evaluation
    num_train_epochs=3,              # Number of training epochs
    weight_decay=0.01,               # Strength of weight decay
    logging_dir=log_dir,            # Directory for storing logs
    logging_steps=10,
    save_total_limit=2,              # Limit the total number of model checkpoints
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # You can split train/eval sets for more robust evaluation
)

# Fine-tune the model
trainer.train()

FailedPreconditionError: c:\Users\maria\OneDrive\Área de Trabalho\Facul\UFPB\P6\NLP\Projeto-Final\Projeto.NLP-2024\logs is not a directory

In [ ]:
# Save the model and tokenizer
# model.save_pretrained("./fine_tuned_recipe_model")
# tokenizer.save_pretrained("./fine_tuned_recipe_model")

## Gerando receitas

In [ ]:
# Load the fine-tuned model
fine_tuned_model = GPT2LMHeadModel.from_pretrained("./fine_tuned_recipe_model")
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("./fine_tuned_recipe_model")

In [3]:
# Define a function to generate a recipe based on ingredients
def generate_recipe(ingredients):
    prompt = f"I have the following ingredients: {ingredients}. What recipe can I make?"
    input_ids = fine_tuned_tokenizer.encode(prompt, return_tensors='pt')

    # Tokenize the input with padding and create an attention mask
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True)
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask  # Attention mask to ignore padding

    # Generate the recipe with the attention mask and pad_token_id properly set
    output_ids = model.generate(
        input_ids, 
        attention_mask=attention_mask,  # Pass attention mask
        max_length=200, 
        num_beams=5, 
        no_repeat_ngram_size=2, 
        early_stopping=True, 
        pad_token_id=tokenizer.pad_token_id  # Properly set the pad_token_id
    )
    
    # Decode the output
    #recipe = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    recipe = fine_tuned_tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    return recipe

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [ ]:
# Create the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Text-to-Recipe Converter")
    gr.Markdown("Enter a list of ingredients and the model will generate a recipe for you.")
    
    ingredients_input = gr.Textbox(label="Enter Ingredients (comma separated)")
    recipe_output = gr.Textbox(label="Generated Recipe")
    
    generate_button = gr.Button("Generate Recipe")
    
    generate_button.click(generate_recipe, inputs=ingredients_input, outputs=recipe_output)

# Launch the interface
demo.launch()